# Urejanje podatkov v `pandas`

V nadaljevanju predavanja bomo skozi sestavljanje podatkovnih tabel `studenti` in `studenti_ocene` spoznali funkcije knjižnice `pandas`, ki jih uporabljamo pri urejanju podatkov iz osnovne tabele `pt`, ki jih zdaj še enkrat uvozimo iz preglednice Excel.

In [1]:
import pandas as pd

vse_pt = pd.read_excel(
    "https://kt.ijs.si/~ljupco/lectures/papvp-2324/rezultati-kolokvijev.xlsx",
    sheet_name = None
)
for list in vse_pt:
    vse_pt[list]["kolokvij"] = list
pt = pd.concat(vse_pt.values())


## Razdruževanje vrednosti

Najprej se bomo lotili problema v stolpcu `spol in starost`, ki združuje vrednosti dveh spremenljivk, spol in starost. V ta namen bomo uporabili funkcijo [`str.split`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html), ki je pravzaprav metoda podatkovnega tipa zaporedje:

In [2]:
print(pt["spol in starost"].str.split("-"))

0    [ž, 19]
1    [m, 20]
2    [m, 19]
3    [ž, 20]
4    [m, 21]
0    [ž, 19]
1    [ž, 21]
2    [m, 19]
3    [ž, 20]
4    [m, 21]
Name: spol in starost, dtype: object


Rezultat funkcije je torej zaporedje seznamov. Če nastavimo argument `expand` te funkcije na vrednost `True`, dobimo kot rezultat podatkovno tabelo z ločenimi stolpci:

In [3]:
print(pt["spol in starost"].str.split("-", expand = True))

   0   1
0  ž  19
1  m  20
2  m  19
3  ž  20
4  m  21
0  ž  19
1  ž  21
2  m  19
3  ž  20
4  m  21


S preprostim prirejanjem tega rezultata in posebnim indeksiranjem, ki smo ga spoznali na prejšnjih predavanjih, ga lahko pretvorimo v dva nova stolpca podatkovne tabele `pt`:

In [4]:
pt[["spol", "starost"]] = pt["spol in starost"].str.split("-", expand = True)
print(pt)

      ime  telefon spol in starost  prvi semester  drugi semester  \
0     Ana      431            ž-19             80              97   
1  Branko      720            m-20             78              74   
2   David      141            m-19             75              68   
3     Eva      210            ž-20             63              82   
4   Franc      592            m-21             95              82   
0     Ana      431            ž-19             82              95   
1  Cvetka      761            ž-21             63              87   
2   David      141            m-19             92              81   
3     Eva      210            ž-20             82              63   
4   Franc      592            m-21             99              54   

         kolokvij spol starost  
0   prvi kolokvij    ž      19  
1   prvi kolokvij    m      20  
2   prvi kolokvij    m      19  
3   prvi kolokvij    ž      20  
4   prvi kolokvij    m      21  
0  drugi kolokvij    ž      19  
1  drugi 

## Tvorjenje in urejanje tabele `studenti`

Zdaj lahko z uporabo indeksiranja preprosto izberemo stolpce, ki tvorijo tabelo `studenti`:

In [43]:
studenti = pt.loc[:, ["ime", "spol", "starost", "telefon"]]
print(studenti)

      ime spol starost  telefon
0     Ana    ž      19      431
1  Branko    m      20      720
2   David    m      19      141
3     Eva    ž      20      210
4   Franc    m      21      592
0     Ana    ž      19      431
1  Cvetka    ž      21      761
2   David    m      19      141
3     Eva    ž      20      210
4   Franc    m      21      592


Ta tabela še vedno po nepotrebnem dva krat ponavlja vrstico za vsako študentko. Na prvi pogled, bi lahko zadevo rešili tako, da izberemo zgolj prvih pet vrstic tabele:

In [44]:
print(pt.loc[[0, 1, 2, 3, 4, 1], ["ime", "spol", "starost", "telefon"]])

      ime spol starost  telefon
0     Ana    ž      19      431
0     Ana    ž      19      431
1  Branko    m      20      720
1  Cvetka    ž      21      761
2   David    m      19      141
2   David    m      19      141
3     Eva    ž      20      210
3     Eva    ž      20      210
4   Franc    m      21      592
4   Franc    m      21      592
1  Branko    m      20      720
1  Cvetka    ž      21      761


Zaradi ponavljajočih se indeksov vrstic ta poskus ni bil uspešen. Lahko bi pa to naredili s pozicijskimi indeksi:

In [46]:
print(studenti.iloc[[0, 1, 2, 3, 4, 6]])

      ime spol starost  telefon
0     Ana    ž      19      431
1  Branko    m      20      720
2   David    m      19      141
3     Eva    ž      20      210
4   Franc    m      21      592
1  Cvetka    ž      21      761


Kar je malo zoprno, ker moramo šteti na roke. Tega seveda ne moremo narediti za velike tabele s podatki.

Pravi pristop sloni na uporabi funkcije oziroma metode podatkovnega tipa podatkovna tabela [`drop_duplicates`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html), ki zbriše podvojene vrstice v podani podatkovni tabeli:

In [47]:
studenti = studenti.drop_duplicates()
print(studenti)

      ime spol starost  telefon
0     Ana    ž      19      431
1  Branko    m      20      720
2   David    m      19      141
3     Eva    ž      20      210
4   Franc    m      21      592
1  Cvetka    ž      21      761


Pri urejanju podatkov običajno poskrbimo, da so vrstice urejene v naraščajočem vrstnem redu glede na vrednost dimenzijske spremenljivke. V ta namen bomo uporabili metodo podatkovnega tipa podatkovna tabela [`sort_values`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html). Njen prvi argument `by` nastavimo na ime stolpca (ali seznam imen stolpcev) za urejanje:

In [49]:
studenti = studenti.sort_values(by = "ime")
print(studenti)

      ime spol starost  telefon
0     Ana    ž      19      431
1  Branko    m      20      720
1  Cvetka    ž      21      761
2   David    m      19      141
3     Eva    ž      20      210
4   Franc    m      21      592


Preverimo zdaj tipe stolpcev v podatkovni tabeli:

In [50]:
print(studenti.dtypes)

ime        object
spol       object
starost    object
telefon     int64
dtype: object


Dva stolpca imata napačna tipa. Stolpec spol bi mogel biti tipa kategorija, stolpec starost pa celoštevilskega tipa. Z indeksiranjem in prirejanjem lahko to uredimo takole (preverite zapiske iz prejšnjih predavanj):

In [54]:
from pandas.api.types import CategoricalDtype

studenti.spol = studenti.spol.astype(CategoricalDtype(studenti.spol.drop_duplicates()))
studenti.starost = studenti.starost.astype("int64")

print(studenti)
print(studenti.dtypes)

      ime spol  starost  telefon
0     Ana    ž       19      431
1  Branko    m       20      720
1  Cvetka    ž       21      761
2   David    m       19      141
3     Eva    ž       20      210
4   Franc    m       21      592
ime          object
spol       category
starost       int64
telefon       int64
dtype: object


Zadnja podrobnost je indeksiranje. Za vajo spremenite indekse vrstic v zadnji tabeli `studenti` na dva načina. Najprej tako, da bodo indeksi vrstic naravna števila od `0` do `5`, brez ponavljanja. Nato pa tako, da so indeksi vrstic imena študentov.

## Tvorjenje in urejanje tabele `studenti_ocene`

Pripravimo zdaj podatke o dosežkih študentov:

In [82]:
studenti_ocene = pt[["ime", "prvi semester", "drugi semester", "kolokvij"]]
print(studenti_ocene)

      ime  prvi semester  drugi semester        kolokvij
0     Ana             80              97   prvi kolokvij
1  Branko             78              74   prvi kolokvij
2   David             75              68   prvi kolokvij
3     Eva             63              82   prvi kolokvij
4   Franc             95              82   prvi kolokvij
0     Ana             82              95  drugi kolokvij
1  Cvetka             63              87  drugi kolokvij
2   David             92              81  drugi kolokvij
3     Eva             82              63  drugi kolokvij
4   Franc             99              54  drugi kolokvij


Zamenjajmo najprej vrednosti v zadnjem stolpcu s celoštevilskimi vrednostmi. Uporabimo funkcijo [`replace`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html):

In [83]:
studenti_ocene = studenti_ocene.replace({
    "prvi kolokvij": 1,
    "drugi kolokvij": 2
})
print(studenti_ocene)

      ime  prvi semester  drugi semester  kolokvij
0     Ana             80              97         1
1  Branko             78              74         1
2   David             75              68         1
3     Eva             63              82         1
4   Franc             95              82         1
0     Ana             82              95         2
1  Cvetka             63              87         2
2   David             92              81         2
3     Eva             82              63         2
4   Franc             99              54         2


Spomnimo se še, da je naslednja težava tabele `studenti_ocene` v tem, da bi imeni spremenljivk `prvi semester` in `drugi semester` morali pravzaprav biti vrednosti spremenljivke `semester`. Za pretvorbo imen stolpcev v spremenljivko uporabljamo funkcijo [`melt`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) z naslednjimi argumenti:

  * `frame` je ime podatkovne tabele, ki jo hočemo spremeniti;

  * `id_vars` je nabor ali seznam imen stolpcev, ki identificirajo vrstice, in katerih vrednosti se ne bodo spreminjale;

  * `value_vars` je nabor ali seznam imen stolpcev, ki predstavljajo podatkovne vrednosti (privzeta vrednost so vsi stolpci `frame`, ki niso v seznamu `id_vars`);

  * `var_name` je ime nove spremenljivke (stolpca), ki prevzame podatkovne vrednosti iz imena stolpca; in

  * `value_name` je ime spremenljivke (stolpca), ki prevzame vrednosti v stolpcih `value_vars`.

In [84]:
studenti_ocene = studenti_ocene.melt(
    id_vars = ["ime", "kolokvij"],
    value_vars = ["prvi semester", "drugi semester"],
    var_name = "semester",
    value_name = "rezultat"
)
print(studenti_ocene)

       ime  kolokvij        semester  rezultat
0      Ana         1   prvi semester        80
1   Branko         1   prvi semester        78
2    David         1   prvi semester        75
3      Eva         1   prvi semester        63
4    Franc         1   prvi semester        95
5      Ana         2   prvi semester        82
6   Cvetka         2   prvi semester        63
7    David         2   prvi semester        92
8      Eva         2   prvi semester        82
9    Franc         2   prvi semester        99
10     Ana         1  drugi semester        97
11  Branko         1  drugi semester        74
12   David         1  drugi semester        68
13     Eva         1  drugi semester        82
14   Franc         1  drugi semester        82
15     Ana         2  drugi semester        95
16  Cvetka         2  drugi semester        87
17   David         2  drugi semester        81
18     Eva         2  drugi semester        63
19   Franc         2  drugi semester        54


Zamenjajmo vrednosti spremenljivke `semester` kot smo to storili zgoraj za `kolokvij`:

In [86]:
studenti_ocene.semester = studenti_ocene.semester.replace({
    "prvi semester": 1,
    "drugi semester": 2
})
print(studenti_ocene)

       ime  kolokvij  semester  rezultat
0      Ana         1         1        80
1   Branko         1         1        78
2    David         1         1        75
3      Eva         1         1        63
4    Franc         1         1        95
5      Ana         2         1        82
6   Cvetka         2         1        63
7    David         2         1        92
8      Eva         2         1        82
9    Franc         2         1        99
10     Ana         1         2        97
11  Branko         1         2        74
12   David         1         2        68
13     Eva         1         2        82
14   Franc         1         2        82
15     Ana         2         2        95
16  Cvetka         2         2        87
17   David         2         2        81
18     Eva         2         2        63
19   Franc         2         2        54


Uredimo stolpce v želenem vrstnem redu in vrstice v naraščajočem vrstnem redu glede na vrednosti dimenzijskih spremenljivk:

In [87]:
studenti_ocene = studenti_ocene[["ime", "semester", "kolokvij", "rezultat"]]
studenti_ocene = studenti_ocene.sort_values(by = ["ime", "semester", "kolokvij"])

print(studenti_ocene)

       ime  semester  kolokvij  rezultat
0      Ana         1         1        80
5      Ana         1         2        82
10     Ana         2         1        97
15     Ana         2         2        95
1   Branko         1         1        78
11  Branko         2         1        74
6   Cvetka         1         2        63
16  Cvetka         2         2        87
2    David         1         1        75
7    David         1         2        92
12   David         2         1        68
17   David         2         2        81
3      Eva         1         1        63
8      Eva         1         2        82
13     Eva         2         1        82
18     Eva         2         2        63
4    Franc         1         1        95
9    Franc         1         2        99
14   Franc         2         1        82
19   Franc         2         2        54


In preverimo tipe:

In [88]:
print(studenti_ocene.dtypes)

ime         object
semester     int64
kolokvij     int64
rezultat     int64
dtype: object


S tipi je načeloma vse v redu. Vseeno za vajo pretvori spremenljivki `semester` in `kolokvij` v kategoriji. Nadalje, za vajo izračunaj domeno (množico ali seznam možnih vrednosti) posamezne spremenljivke v tabeli.

Zadnja stvar, ki še ostaja neurejena, je, da podatkovna tabela (in pripadajoča preslikava) ne pokriva vseh možnih kombinacij vrednosti dimenzijskih spremenljivk. Tako, na primer, ne vemo kakšen je bil dosežek Branka na drugem kolokviju v prvem semestru. Domnevamo lahko, da Branko ni pristopil k temu kolokviju in zato nima ocene. Bolj korekten zapis tega dejstva bi bila nova vrstica v tabeli `studenti_ocene` z vrednostmi `Branko`, `1`, `2` in `NaN`. Tako bomo prepričani, da pomanjkanje vrstice ni posledica napake pri vnosu ali prenosu podatkov.

Zato, da dosežemo zahtevno dopolnjevanje podatkovne tabele, moramo najprej spoznati nekaj osnov poizvedovalnega jezika SQL.

## Združevanje tabel v `pandas` in poizvedovalni jezik SQL

SQL (_Structured Query Language_) je posebna **vrsta programskega jezika za poizvedovanje po podatkih** shranjenih v podatkovnih bazah. Ker so podatki v bazah običajno shranjeni v večjem številu tabel, nam SQL ponuja številne možnosti za združevanje podatkov iz različnih tabel v eno in operacije, ki obravnavajo tabele kot množice, katerih elementi ustrezajo vrsticam v tabelah. V nadaljevanju tega poglavja bomo videli zakaj nam to koristi pri urejanju tabel za analizo podatkov.

Združevanje tabel v jeziku SQL opravi operacija JOIN. Operacijo v knjižnici `pandas` opravlja funkcija [`merge`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html). Funkcija združi dve tabeli tako, da naredi kartezični produkt vrstic prve (leve) in druge (desne) tabele. Število stolpcev združene tabele ima toliko stolpcev, kot je seštevek števila stolpcev v prvi in drugi tabeli. Taka operacija bi bila sicer omejeno uporabna, ker JOIN uporabljamo takrat, ko hočemo v kartezičnem produktu ohraniti zgolj tiste vrstice, ki imajo isto vrednost skupnih stolpcev v obeh tabelah.

Preden si ogledamo primer, ki bodo razjasnile pomen opisa zgoraj, poglejmo argumente funkcije `merge`.

  * `left` podaja ime prve (leve) podatkovne tabele za združevanje.
  
  * `right` podaja ime druge (desne) podatkovne tabele za združevanje.

  * `on` podaja ime ali seznam imen _izbranih_ spremenljivk (stolpcev), katerih vrednosti v združeni tabeli hočemo, da so enake.
  
  * `how` določa način združevanja in je lahko `left`, `right`, `outer`, `inner` ali `cross`:

    * `left` v združeno tabelo upošteva vrednosti _izbranih_ spremenljivk iz leve tabele;

    * `right` v združeno tabelo upošteva vrednosti _izbranih_ spremenljivk iz desne tabele;

    * `inner` v združeno tabelo upošteva zgolj _presek_ množic vrednosti _izbranih_ spremenljivk iz leve in desne tabele;

    * `outer` v združeno tabelo upošteva zgolj _unijo_ množic vrednosti _izbranih_ spremenljivk iz leve in desne tabele; in
    
    * `cross` v združeni tabeli naredi kartezični produkt podanih podatkovnih tabel.

Naredimo zdaj dve tabeli in poglejmo rezultate različnih načinov združevanja, kjer bo izbrana spremenljivka `id`.

In [89]:
dt_x = pd.DataFrame({
  "id": pd.Series([1, 1, 2, 2, 3]),
  "vrednost": pd.Series(["x11", "x12", "x21", "x22", "x3"])
})
print(dt_x)

dt_y = pd.DataFrame({
  "id": pd.Series([1, 2, 2, 4, 4]),
  "vrednost": pd.Series(["y1", "y21", "y22", "y41", "y42"])
})
print(dt_y)

   id vrednost
0   1      x11
1   1      x12
2   2      x21
3   2      x22
4   3       x3
   id vrednost
0   1       y1
1   2      y21
2   2      y22
3   4      y41
4   4      y42


Levo združevanje ohrani vse vrstice iz leve tabele `dt_x`. Če v desni tabeli ni vrstice z vrednostjo izbrane spremenljivke iz leve tabele, potem je vrednost `NaN`.

In [90]:
print(dt_x.merge(dt_y, on = "id", how = "left"))

   id vrednost_x vrednost_y
0   1        x11         y1
1   1        x12         y1
2   2        x21        y21
3   2        x21        y22
4   2        x22        y21
5   2        x22        y22
6   3         x3        NaN


Desno združevanje naredi obratno in ohrani vse vrstice iz desne tabele:

In [91]:
print(dt_x.merge(dt_y, on = "id", how = "right"))

   id vrednost_x vrednost_y
0   1        x11         y1
1   1        x12         y1
2   2        x21        y21
3   2        x22        y21
4   2        x21        y22
5   2        x22        y22
6   4        NaN        y41
7   4        NaN        y42


Poglejmo še `inner` in `outer`:

In [92]:
print(dt_x.merge(dt_y, on = "id", how = "inner"))
print(dt_x.merge(dt_y, on = "id", how = "outer"))

   id vrednost_x vrednost_y
0   1        x11         y1
1   1        x12         y1
2   2        x21        y21
3   2        x21        y22
4   2        x22        y21
5   2        x22        y22
   id vrednost_x vrednost_y
0   1        x11         y1
1   1        x12         y1
2   2        x21        y21
3   2        x21        y22
4   2        x22        y21
5   2        x22        y22
6   3         x3        NaN
7   4        NaN        y41
8   4        NaN        y42


Po pričakovanjih, podatkovna tabela združena z `inner` nima neznanih vrednosti, saj upošteva zgolj vrednosti izbrane spremenljivke `id`, ki so skupne obema tabelama. Podatkovna tabela združena z `outer` pa vključuje vse vrednosti spremenljivke `id` in zato vpelje tri neznane vrednosti.

Nenazadnje združevanje tipa `cross` vrne kartezični produkt vrstic leve in desne tabele, ne da bi pri tem upoštevali izbrane skupne spremenljivke:

In [94]:
print(dt_x.merge(dt_y, how = "cross"))

    id_x vrednost_x  id_y vrednost_y
0      1        x11     1         y1
1      1        x11     2        y21
2      1        x11     2        y22
3      1        x11     4        y41
4      1        x11     4        y42
5      1        x12     1         y1
6      1        x12     2        y21
7      1        x12     2        y22
8      1        x12     4        y41
9      1        x12     4        y42
10     2        x21     1         y1
11     2        x21     2        y21
12     2        x21     2        y22
13     2        x21     4        y41
14     2        x21     4        y42
15     2        x22     1         y1
16     2        x22     2        y21
17     2        x22     2        y22
18     2        x22     4        y41
19     2        x22     4        y42
20     3         x3     1         y1
21     3         x3     2        y21
22     3         x3     2        y22
23     3         x3     4        y41
24     3         x3     4        y42


## Dokončno urejanje tabele `studenti_ocene`

Zdaj imamo v rokah vsa orodja za dokončno urejanje tabele `studenti_ocene` z dodajanjem manjkajočih vrstic, tako, da bo podatkovna tabela vsebovala vrstico za vse možne kombinacije vrednosti dimenzijskih spremenljivk. Najprej bomo izračunali tabelo vseh možnih kombinacij tako, da bomo izračunali kartezični produkt domen dimenzijskih spremenljivk:

In [102]:
d_ime = pd.DataFrame({
    "ime": studenti_ocene.ime.drop_duplicates()
})
d_semester = pd.DataFrame({
    "semester": studenti_ocene.semester.drop_duplicates()
})
d_kolokvij = pd.DataFrame({
    "kolokvij": studenti_ocene.kolokvij.drop_duplicates()
})

d_vse = d_ime.merge(d_semester, how = "cross").merge(d_kolokvij, how = "cross")
print(d_vse)

       ime  semester  kolokvij
0      Ana         1         1
1      Ana         1         2
2      Ana         2         1
3      Ana         2         2
4   Branko         1         1
5   Branko         1         2
6   Branko         2         1
7   Branko         2         2
8   Cvetka         1         1
9   Cvetka         1         2
10  Cvetka         2         1
11  Cvetka         2         2
12   David         1         1
13   David         1         2
14   David         2         1
15   David         2         2
16     Eva         1         1
17     Eva         1         2
18     Eva         2         1
19     Eva         2         2
20   Franc         1         1
21   Franc         1         2
22   Franc         2         1
23   Franc         2         2


Po pričakovanjih, ima tabela s kartezičnim produktom 24 vrstic (za vajo ugotovi zakaj oziroma kako na osnovi vrednosti dimenzijskih spremenljivk v tabeli `studenti_ocene` pridemo do tega števila vrstic).

Zdaj lahko dopolnimo tabelo `studenti_ocene` tako, da jo s klicem funkcije `merge` združimo z zgornjo tabelo in tako torej tabeli s kartezičnim produktom dodamo rezultate kolokvijev:

In [105]:
studenti_ocene = d_vse.merge(studenti_ocene, how = "left")
print(studenti_ocene)

       ime  semester  kolokvij  rezultat
0      Ana         1         1      80.0
1      Ana         1         2      82.0
2      Ana         2         1      97.0
3      Ana         2         2      95.0
4   Branko         1         1      78.0
5   Branko         1         2       NaN
6   Branko         2         1      74.0
7   Branko         2         2       NaN
8   Cvetka         1         1       NaN
9   Cvetka         1         2      63.0
10  Cvetka         2         1       NaN
11  Cvetka         2         2      87.0
12   David         1         1      75.0
13   David         1         2      92.0
14   David         2         1      68.0
15   David         2         2      81.0
16     Eva         1         1      63.0
17     Eva         1         2      82.0
18     Eva         2         1      82.0
19     Eva         2         2      63.0
20   Franc         1         1      95.0
21   Franc         1         2      99.0
22   Franc         2         1      82.0
23   Franc      

Vidimo lahko, da ima sedaj tabela `studenti_ocene` vseh 24 vrstic, ki ustrezajo vsem možnim kombinacijam vrednosti njenih dimenzijskih spremenljivk. S tem je urejanje podatkov o rezultatih kolokvijev končano. Tabeli `studenti` in `studenti_ocene` sta pripravljeni za podatkovno analizo in vizualizacijo.

## Naloge

1. V datoteki [`izidi.xlsx`](https://kt.ijs.si/~ljupco/lectures/papvp-2324/izidi.xlsx) je preglednica s podatki o numeričnih izidih uporabe dveh različnih terapij na treh pacientih. Prazne celice ustrezajo neznanim vrednostim izidov. Napiši program, ki prebere podatke iz preglednice in iz njih sestavi podatkovno tabelo z urejenimi podatki. Namig: rabiš sestaviti eno tabelo z dvema dimenzijskima in eno merjeno spremenljivko.

1. V nekem bistroju poimenovanem Mafija strežejo tri vrste kavnih napitkov: črno kavo, kavo z mlekom in kavo s smetano. Pri izdelavi letne bilance si zapišejo koliko skodelic vsake vrste kavnega napitka so postregli v opazovanem letu. Spodnja tabela podaja podatke o petih letnih bilancah bistroja Mafija:

    | Vrsta kavnega napitka | 2018 | 2019 | 2020 | 2021 | 2022 |
    |:---|---:|---:|---:|---:|---:|
    | Črna kava | 23,100 | 3,500 | 6,200 | 18,700 | 21,400 |
    | Kava z mlekom | 18,300 | 2,800 | 4,900 | 14,000 | 18,000 |
    | Kava s smetano | 11,500 | 1,600 | 3,400 | 7,600 | 11,100 |

    Premisli kako bi iz zgornje tabele sestavili podatkovne tabele urejenih podatkov. Sestavi ustrezne podatkovne tabele v `pandas`.


1. Statistični urad Slovenije v svoji podatkovni bazi [SiStat](https://pxweb.stat.si/) ponuja podatke o selitvenem gibanju prebivalstva v Sloveniji med regijami in tujino. Bazo teh podatkov lahko najdemo v področju _Regionalni pregled_ pod rubriko _Statistične regije_ in _Prebivalstvo - statistične regije_. Podatki iz te baze so na voljo v preglednici [`05I2004S_20221025-231704.xlsx`](https://kt.ijs.si/~ljupco/lectures/papvp-2324/05I2004S_20221025-231704.xlsx). Napiši program, ki sestavi urejeno tabelo s podatki o selitvenem gibanju prebivalstva v Sloveniji.

1. Spletna stran [Covid-19 sledilnik](https://covid-19.sledilnik.org/) ponuja dnevno osvežene podatke o stanju epidemije Covid-19 v Sloveniji. Podatki o dnevnem številu okužb ponujajo tudi skozi spletne storitve [GitHub](https://github.com/sledilnik/data/blob/master/csv/stats.csv): ti podatki so v obliki CSV dostopni na naslovu [https://raw.githubusercontent.com/sledilnik/data/master/csv/stats.csv](https://raw.githubusercontent.com/sledilnik/data/master/csv/stats.csv).

    Napiši program, ki iz teh podatkov sestavi podatkovno tabelo z urejenimi dnevnimi podatki o številu okužb v posameznih regijah in celotni Sloveniji. Podatkovna tabela naj vsebuje naslednje tri spremenljivke:
    
    * `datum` je dimenzijska spremenljivka z datumom opazovanega števila okužb.
    
    * `regija` je dimenzijska spremenljivka, ki poda statistično regijo v kateri opazujemo število okužb. Domena spremenljivke naj vključuje vseh [statističnih regij Slovenije](https://sl.wikipedia.org/wiki/Statisti%C4%8Dne_regije_Slovenije) ter Slovenijo kot celoto.
    
    * `kumulativne_okuzbe` je merjena spremenljivka, ki za izbrano regijo poda kumulatinvo število okužb Covid-19 do izbranega datuma.

1. Statistični urad Slovenije v svoji podatkovni bazi [SiStat](https://pxweb.stat.si/) ponuja podatke o številu prebivalcev v posameznih statističnih regijah in celotni Sloveniji po polletjih. Poišči te podatke v bazi in jih izvozi v obliko preglednice Excel: pri tem upoštevaj vse razpoložljive podatke od prvega polletja 2020. Nato jih uredi v podatkovno tabelo s stolpci `polletje`, `regija` in `prebivalstvo`.

    Uporabi podatke iz te tabele in tabele sestavljene v prejšnji nalogi za izračun _deleža_ okuženih prebivalcev v posamezni regiji in celotni Sloveniji.